# Информационный поиск

Скачиваем классический набор данных -- набор текстов об аэронавтике CRANFIELD

In [1]:
# ! wget -q http://ir.dcs.gla.ac.uk/resources/test_collections/cran/cran.tar.gz
# ! tar -xvf cran.tar.gz
# ! rm cran.tar.gz*

'wget' is not recognized as an internal or external command,
operable program or batch file.
tar: Error opening archive: Failed to open 'cran.tar.gz'
'rm' is not recognized as an internal or external command,
operable program or batch file.


Берём только сами запросы (это будут наши документы)

In [3]:
# ! grep -v "^\." cran.qry > just.qry
# ! head -3 just.qry

'Select-String' is not recognized as an internal or external command,
operable program or batch file.
'Select' is not recognized as an internal or external command,
operable program or batch file.


Объединяем многострочные в один

In [7]:
raw_query_data = [line.strip() for line in open("../data/W7T1/just.qry", "r", encoding='utf-16-le').readlines()]
query_data = [""]

for query_part in raw_query_data:
  query_data[-1] += query_part + " "
  if query_part.endswith("."):
    query_data.append("")

query_data[:2] #Выведем пару документов для примера

### Составим запросы к нашим документам

In [28]:
QUERIES = ['delta wings.', 'dissociated free stream.']

## Boolean retrieval
Представим каждый документ как "битовую маску": вектор размером со словарь, в котором на каждой позиции единица, если в документе есть соответсвующий терм, и ноль, если терма нет

In [ ]:
# в разных версиях ответы могут отличаться, поэтому важно иметь одну и ту же
# ! pip install -q scikit-learn==0.22.2.post1

In [29]:
from  sklearn.feature_extraction.text import CountVectorizer

encoder = CountVectorizer(binary=True)
encoded_data = encoder.fit_transform(query_data)
encoded_queries = encoder.transform(QUERIES)
list(encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

Посмотрим на представление первого предложения

In [30]:
id2term = {idx: term for term, idx in encoder.vocabulary_.items()}
non_zero_values_ids = encoded_data[0].nonzero()[1]

terms = [id2term[idx] for idx in non_zero_values_ids]
terms

['what',
 'similarity',
 'laws',
 'must',
 'be',
 'obeyed',
 'when',
 'constructing',
 'aeroelastic',
 'models',
 'of',
 'heated',
 'high',
 'speed',
 'aircraft']

Всё так.

## Задание 0

Теперь для каждого из данных запросов `QUERIES` найдём ближайший для него документ из `query_data` по сходству Жаккара. Есть более эффективные способы это сделать, но вам требуется реализовать расстояние Жаккара и далее применить его к нашим данным.

In [35]:
import numpy as np 

def jaccard_sim(vector_a: np.array, vector_b: np.array) -> float:
  """
    Сходство или коэффициент Жаккара: отношение мощности пересечения
    к мощности объединения
  """
  return float(sum(vector_a & vector_b) / sum(vector_a | vector_b))

#Проверка, что функция работает правильно
assert jaccard_sim(np.array([1, 0, 1, 0, 1]), np.array([0, 1, 1, 1, 1])) == 0.4

Здесь документы представлены так же, как строки в матрице термов-документов. Каждая ячейка вектора отвечает за наличие/отсутствие конкретного элемента (например, слова-терма, когда у нас в словаре всего 5 слов). В первом случае их три, во втором — четыре. Объединение — все пять возможных элементов. Пересечение — два. Отсюда и 0.4.

## Задание 1
Теперь с помощью кода ниже вычислите для каждого запроса самые близкие документы.

In [36]:
for q_id, query in enumerate(encoded_queries):
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in encoded_data]
  # вычисляем коэфф. Жаккара
  id2doc2similarity = [(doc_id, doc, jaccard_sim(query, doc)) for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=True)

  print("Q: %s\nFOUND:" % QUERIES[q_id])
  # выводим по 3 наиболее близких документа для каждого запроса
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))



Q: delta wings.
FOUND:
    29	0.12	papers on flow visualization on slender conical wings . 
    28	0.11	what is the effect of cross sectional shape on the flow over simple delta wings with sharp leading edges . 
    27	0.09	what application has the linear theory design of curved wings . 
Q: dissociated free stream.
FOUND:
    73	0.14	how significant is the possible pressure of a dissociated free stream with respect to the realization of hypersonic simulation in high enthalpy wind tunnels . 
    157	0.09	have flow fields been calculated for blunt-nosed bodies and compared with experiment for a wide range of free stream conditions and body shapes . 
    32	0.05	how do interference-free longitudinal stability measurements (made using free-flight models) compare with similar measurements made in a low-blockage wind tunnel . 


Видим, что кое-где просачиваются  тексты, которых с запросами объединяют малозначительные термы, но при этом коэффициент Жаккара -- наша функция ранжирования! -- высок.

# VSM

Попробуем теперь сделать то же, но с tf-idf и косинусным расстоянием. Мы сделаем всё опять "руками", но "в реальной жизни" лучше использоватьесть эффективные реализации cosine distance, например, из библиотеки scipy.

In [37]:
from  sklearn.feature_extraction.text import TfidfVectorizer

# Совет: обязательно разберитесь с тем, какие возможности
# предоставляет tf-idf vectorizer, какие параметры за что отвечают

tfidf_encoder = TfidfVectorizer()
tfidf_encoded_data = tfidf_encoder.fit_transform(query_data)
tfidf_encoded_queries = tfidf_encoder.transform(QUERIES)

list(tfidf_encoder.vocabulary_)[:3]

['what', 'similarity', 'laws']

## Задание 2
Реализовать косинусное расстояние

In [34]:
import numpy as np 

def cosine_distance(vector_a: np.array, vector_b: np.array) -> float:
  """
    Косинусное расстояние: единица минус отношение скалярного произведения
    на произведение L2-норм (подсказка: в numpy такие нормы есть)
  """
  # ваш код здесь

  scalar_prod = np.dot(vector_a, vector_b)
  vector_a_L2 = np.linalg.norm(vector_a, 2)
  vector_b_L2 = np.linalg.norm(vector_b, 2)
  return  1-scalar_prod/(vector_a_L2*vector_b_L2)
#Проверка, что функция работает правильно
assert cosine_distance(np.array([1, 0, 1, 1, 1]), np.array([0, 0, 1, 0, 0])) == 0.5


Теперь вычислим ближайшие по косинусному расстоянию между векторными представлениями документов и запросов

In [26]:
for q_id, query in enumerate(tfidf_encoded_queries):
  
  # приводим к нужному типу
  query = query.todense().A1
  docs = [doc.todense().A1 for doc in tfidf_encoded_data]
  # Косинусное расстояние
  id2doc2similarity = [(doc_id, doc, cosine_distance(query, doc)) \
                       for doc_id, doc in enumerate(docs)]
  # сортируем по нему
  closest = sorted(id2doc2similarity, key=lambda x: x[2], reverse=False)
  
  print("Q: %s\nFOUND:" % QUERIES[q_id])
  
  for closest_id, _, sim in closest[:3]:
    print("    %d\t%.2f\t%s" %(closest_id, sim, query_data[closest_id]))

Q: delta wings.
FOUND:
    28	0.60	what is the effect of cross sectional shape on the flow over simple delta wings with sharp leading edges . 
    29	0.74	papers on flow visualization on slender conical wings . 
    27	0.77	what application has the linear theory design of curved wings . 
Q: dissociated free stream.
FOUND:
    73	0.55	how significant is the possible pressure of a dissociated free stream with respect to the realization of hypersonic simulation in high enthalpy wind tunnels . 
    157	0.75	have flow fields been calculated for blunt-nosed bodies and compared with experiment for a wide range of free stream conditions and body shapes . 
    171	0.80	has a comparison been made between interference-free drag measurements using free-flight models and similar measurements made in a low-blockage wind tunnel . 


C:\Users\mikhail.komarov\AppData\Local\Temp\ipykernel_14820\645080490.py:13: RuntimeWarning: invalid value encountered in double_scalars
  return  1-scalar_prod/(vector_a_L2*vector_b_L2)
